In [1]:
# %% [markdown]
# Snapshot GraphQL sanity check for one proposal
# - Fetch proposal meta (with start/end)
# - Fetch proposal result (with start/end)
# - Fetch all votes (paginated)
# - Save to ./snapshot_dump/<proposal_id>/
# Notes:
# - This bypasses MCP to validate data availability directly from Snapshot hub.
# - If this succeeds but your MCP still fails, fix the MCP GraphQL query fields.

# %%
import os, re, json, time, math, pathlib, datetime, typing as t
import requests

SNAPSHOT_API = os.environ.get("SNAPSHOT_API", "https://hub.snapshot.org/graphql")
TIMEOUT = int(os.environ.get("SNAPSHOT_TIMEOUT", "30"))

# Target proposal URL (you can edit here)
SNAPSHOT_URL = "https://snapshot.box/#/s:aavedao.eth/proposal/0xa2b9d0717a82a111acc27e514bed07caa9b8636c12dd68fb61ae4dc57503c3cd"

# ------------- Helpers -------------
def normalize_url(u: str) -> str:
    """Normalize snapshot.box s: prefix to snapshot.org canonical form."""
    v = u.replace("/#/s:", "/#/")
    v = re.sub(r"^https://snapshot\.box", "https://snapshot.org", v)
    return v

def extract_proposal_id(u: str) -> t.Optional[str]:
    """Extract 66-hex-char id from URL."""
    m = re.search(r"/proposal/(0x[a-fA-F0-9]{64})", u)
    return m.group(1) if m else None

def iso_from_unix(ts: t.Optional[int]) -> t.Optional[str]:
    """Convert unix seconds to UTC ISO8601 string."""
    if ts is None:
        return None
    try:
        return datetime.datetime.utcfromtimestamp(int(ts)).strftime("%Y-%m-%dT%H:%M:%SZ")
    except Exception:
        return None

def gql(query: str, variables: dict) -> dict:
    """POST GraphQL with basic retry/backoff."""
    sess = requests.Session()
    sess.headers.update({"User-Agent": "snapshot-graph-check/1.0"})
    retries = 0
    while True:
        try:
            r = sess.post(
                SNAPSHOT_API,
                json={"query": query, "variables": variables},
                timeout=TIMEOUT,
            )
            if r.status_code == 200:
                return r.json()
            if r.status_code in (429, 502, 503, 504) and retries < 6:
                delay = 1.5 ** retries
                time.sleep(delay + (0.1 * math.sin(retries)))
                retries += 1
                continue
            r.raise_for_status()
        except requests.RequestException:
            if retries < 6:
                delay = 1.5 ** retries
                time.sleep(delay + (0.1 * math.cos(retries)))
                retries += 1
                continue
            raise

# ------------- GraphQL queries (include start/end explicitly) -------------
Q_PROPOSAL_BY_ID = """
query ($id: String!) {
  proposal(id: $id) {
    id
    title
    body
    author
    choices
    start
    end
    discussion
    state
    space {
      id
      name
      strategies { name network params }
    }
  }
}
"""

Q_PROPOSAL_RESULT = """
query($id: String!) {
  proposal(id: $id) {
    id
    choices
    scores
    scores_total
    state
    start
    end
  }
}
"""

Q_VOTES_PAGE = """
query($proposal: String!, $first: Int!, $skip: Int!) {
  votes(
    first: $first
    skip: $skip
    where: { proposal: $proposal }
    orderBy: "created"
    orderDirection: asc
  ) {
    id
    voter
    created
    choice
    vp
    reason
  }
}
"""

# ------------- Resolve proposal id -------------
norm_url = normalize_url(SNAPSHOT_URL)
pid = extract_proposal_id(norm_url)
if not pid:
    raise SystemExit(f"Cannot extract proposal id from URL: {SNAPSHOT_URL}")

print("Proposal ID:", pid)

# ------------- Fetch proposal meta -------------
p_resp = gql(Q_PROPOSAL_BY_ID, {"id": pid})
proposal = (p_resp.get("data") or {}).get("proposal") or {}
if not proposal:
    raise SystemExit("Proposal not found. Check id or network availability.")

print("Meta fields present:", sorted(proposal.keys()))
print("start:", proposal.get("start"), "end:", proposal.get("end"))

# ------------- Fetch proposal result (contains start/end too) -------------
r_resp = gql(Q_PROPOSAL_RESULT, {"id": pid})
result = (r_resp.get("data") or {}).get("proposal") or {}
print("Result fields present:", sorted(result.keys()))
print("result.start:", result.get("start"), "result.end:", result.get("end"))

# ------------- Fetch ALL votes (paged) -------------
all_votes = []
skip = 0
batch = 500
while True:
    v_resp = gql(Q_VOTES_PAGE, {"proposal": pid, "first": batch, "skip": skip})
    page = (v_resp.get("data") or {}).get("votes") or []
    if not page:
        break
    all_votes.extend(page)
    if len(page) < batch:
        break
    skip += batch

print("Total votes fetched:", len(all_votes))

# ------------- Persist to disk -------------
outdir = pathlib.Path("snapshot_dump") / pid
outdir.mkdir(parents=True, exist_ok=True)

with (outdir / "proposal.json").open("w", encoding="utf-8") as fh:
    json.dump(proposal, fh, ensure_ascii=False, indent=2)

with (outdir / "result.json").open("w", encoding="utf-8") as fh:
    json.dump(result, fh, ensure_ascii=False, indent=2)

with (outdir / "votes.jsonl").open("w", encoding="utf-8") as fh:
    for v in all_votes:
        fh.write(json.dumps(v, ensure_ascii=False) + "\n")

combined = {
    "proposal_id": pid,
    "snapshot_url_normalized": norm_url,
    "meta": {
        "title": proposal.get("title"),
        "space": (proposal.get("space") or {}).get("id"),
        "choices": proposal.get("choices"),
        "start_unix": proposal.get("start"),
        "end_unix": proposal.get("end"),
        "end_iso": iso_from_unix(proposal.get("end")),
        "state": proposal.get("state"),
        "discussion": proposal.get("discussion"),
    },
    "result": {
        "scores": result.get("scores"),
        "scores_total": result.get("scores_total"),
        "state": result.get("state"),
        "start_unix": result.get("start"),
        "end_unix": result.get("end"),
        "end_iso": iso_from_unix(result.get("end")),
    },
    "counts": {"votes": len(all_votes)},
}
with (outdir / "combined.json").open("w", encoding="utf-8") as fh:
    json.dump(combined, fh, ensure_ascii=False, indent=2)

print(f"\nSaved to: {outdir.resolve()}")
print(" - proposal.json")
print(" - result.json")
print(" - votes.jsonl")
print(" - combined.json")
print("\nEvent end ISO (from meta or result):",
      combined["meta"]["end_iso"] or combined["result"]["end_iso"])


Proposal ID: 0xa2b9d0717a82a111acc27e514bed07caa9b8636c12dd68fb61ae4dc57503c3cd
Meta fields present: ['author', 'body', 'choices', 'discussion', 'end', 'id', 'space', 'start', 'state', 'title']
start: 1757151443 end: 1757410643
Result fields present: ['choices', 'end', 'id', 'scores', 'scores_total', 'start', 'state']
result.start: 1757151443 result.end: 1757410643
Total votes fetched: 103

Saved to: /Users/chunghyunhan/Projects/agentics/snapshot_dump/0xa2b9d0717a82a111acc27e514bed07caa9b8636c12dd68fb61ae4dc57503c3cd
 - proposal.json
 - result.json
 - votes.jsonl
 - combined.json

Event end ISO (from meta or result): 2025-09-09T09:37:23Z


/var/folders/vz/g2dffj7n39s65g4gwshdmsbc0000gn/T/ipykernel_33930/2178554158.py:38: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.datetime.utcfromtimestamp(int(ts)).strftime("%Y-%m-%dT%H:%M:%SZ")
